In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

### Load in data

In [2]:
family = pd.read_csv('Data/familyxx.csv')
family.head()

,FINT_Y_P,FINT_M_P,FMX,RECTYPE,SRVY_YR,HHX,FM_SIZE,FM_STRCP,FM_TYPE,FM_STRP,...,COVCONF,FHICOST,FMEDBILL,FMEDBPAY,FMEDBNOP,FSAF,FHICOVCT,FHICOVYN,FPRCOOH,FHIEBCCT
0,2018,1,1,60,2018,1,1,11,1,11,...,NaN,1,2,2,NaN,2,1,1,NaN,NaN
1,2018,1,1,60,2018,4,3,41,4,41,...,4.0,2,1,1,1.0,2,3,1,2.0,3.0
2,2018,2,1,60,2018,6,4,41,4,41,...,1.0,2,2,2,NaN,1,4,1,2.0,4.0
3,2018,3,1,60,2018,8,3,41,4,41,...,3.0,3,2,1,NaN,2,3,1,2.0,3.0
4,2018,2,1,60,2018,9,1,11,1,11,...,NaN,2,1,1,1.0,2,1,1,2.0,0.0


In [3]:
person = pd.read_csv('Data/personsx.csv')
person.head()

,FPX,AGE_CHG,INTV_QRT,SEX,NOWAF,FSPOUS2,COHAB1,COHAB2,FCOHAB3,ASTATFLG,...,EDUC1,ERNYR_P,ARMFTM7P,ARMFTM1P,ARMFTM2P,ARMFTM3P,ARMFTM4P,ARMFTM5P,ARMFTM6P,ENGLANG
0,1,NaN,1,2,NaN,NaN,NaN,NaN,NaN,1.0,...,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1,NaN,1,2,2.0,2.0,NaN,NaN,NaN,2.0,...,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,2,NaN,1,1,2.0,1.0,NaN,NaN,NaN,0.0,...,15,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,3,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,1,NaN,1,1,2.0,2.0,NaN,NaN,NaN,1.0,...,21,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [4]:
household = pd.read_csv('Data/househld.csv')
# household.head()

In [5]:
adult = pd.read_csv('Data/samadult.csv')
# adult.head()

In [6]:
child = pd.read_csv('Data/samchild.csv')
# child.head()

Having issues with merging --> possibly a file size issue, only working with smaller portions of each dataset

In [7]:
# merge family and person
# merged = pd.merge(person,family,on='FMX',how='outer')
merged = person.merge(family, on = 'FMX', how='left')

MemoryError: Unable to allocate 16.1 GiB for an array with shape (2156872938,) and data type int64

In [ ]:
merged.describe()

In [ ]:
merged.info()

### Income related factors

In [ ]:
income = merged[['WRKHRS2', 'WRKFTALL', 'WRKLYR1', 'WRKMYR', 'ERNYR_P', 'HIEMPOF', 'PSAL', 'PSEINC', 'PHCDN2WP', 'P10DVYR', 'PHCPN2WP', 'PHCDV2W', 'PHCHM2W']]
# healthcare visit related --> 'PHCDN2WP', 'P10DVYR', 'PHCPN2WP', 'PHCDV2W', 'PHCHM2W'
# income realted --> 'WRKHRS2', 'WRKFTALL', 'WRKLYR1', 'WRKMYR', 'ERNYR_P', 'HIEMPOF', 'PSAL', 'PSEINC'

income

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(income.corr());

In [ ]:
income.isnull().values.any()

In [ ]:
# all columns have null values --> how should i deal with null values?
for i in income.columns:
    if(income[i].isnull().values.any()):
        print(i + '  True  ' + str(income[i].isnull().sum()))
    else:
        print(i + '  False')

In [ ]:
# fill null values with mean --> is this a good approach or should I take a different one?

pd.set_option('mode.chained_assignment', None)
for i in income.columns:
    income[i] = income[i].fillna(round(income[i].mean()))

In [ ]:
# remove values 7, 8, 9 (refused, not ascertained, don't know)
income = income[income['PHCHM2W'] < 7]
income.PHCHM2W.unique()

In [ ]:
income

In [ ]:
X = income[['WRKHRS2', 'WRKFTALL', 'WRKLYR1', 'WRKMYR', 'ERNYR_P', 'HIEMPOF', 'PSAL', 'PSEINC']]
y = income['PHCHM2W']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

from sklearn.preprocessing import StandardScaler as ss
sc = ss()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_pred, y_test)

y_pred_train = classifier.predict(X_train)
cm_train = confusion_matrix(y_pred_train, y_train)

print()
print('Accuracy for training set for Random Forest = {}'.format((cm_train[0][0] + cm_train[1][1])/len(y_train)))
print('Accuracy for test set for Random Forest = {}'.format((cm_test[0][0] + cm_test[1][1])/len(y_test)))

In [ ]:
# sort by highest feature importance
sorted_idx = classifier.feature_importances_.argsort()
plt.barh(X.columns[sorted_idx], classifier.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance")
classifier.feature_importances_[sorted_idx]

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
factors = merged[['PLIMANY', 'LAHCC6', 'LAHCC7A', 'LAHCC8', 'LAHCC10', 'LAHCC11', 'PHCDN2WP', 'P10DVYR', 'PHCPN2WP', 'PHCDV2W', 'PHCHM2W']]
# factors --> 'PLIMANY', 'LAHCC6', 'LAHCC7A', 'LAHCC8', 'LAHCC10', 'LAHCC11'
# healthcare visit --> 'PHCDN2WP', 'P10DVYR', 'PHCPN2WP', 'PHCDV2W', 'PHCHM2W'

In [ ]:
status_vars = merged[['LCCHRC1', 'LCCHRC2', 'LCCHRC3', 'LCCHRC4', 'LCCHRC5', 'LCCHRC6', 'LCCHRC8', 'LCCHRC9', 'LCCHRC10', 'LCCHRC11', 'LCCHRC12', 'LCCHRC13', 'LCCHRC90', 'LCCHRC91', 'PHCDN2WP', 'P10DVYR', 'PHCPN2WP', 'PHCDV2W', 'PHCHM2W']]
# healthcare visit --> 'PHCDN2WP', 'P10DVYR', 'PHCPN2WP', 'PHCDV2W', 'PHCHM2W'
# status variables --> 'LCCHRC1', 'LCCHRC2', 'LCCHRC3', 'LCCHRC4', 'LCCHRC5', 'LCCHRC6', 'LCCHRC8', 'LCCHRC9', 'LCCHRC10', 'LCCHRC11', 'LCCHRC12', 'LCCHRC13', 'LCCHRC90', 'LCCHRC91'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(factors.corr());

In [ ]:
sns.heatmap(status_vars.corr());

In [ ]:
status_vars

In [ ]:
merged['LCCHRC1']

In [ ]:
for i in ['LCCHRC1']:
    if(merged[i].isnull().values.any()):
        print(i + '  True  ' + str(merged[i].isnull().sum()))
    else:
        print(i + '  False')

In [ ]:
for i in ['LCCHRC1']:
    if(person[i].isnull().values.any()):
        print(i + '  True  ' + str(person[i].isnull().sum()))
    else:
        print(i + '  False')

In [ ]:
person

In [ ]:
# family of 4 percent insured
X = family_4['FM_SIZE']
y = family_4['percent_insured']
y.value_counts().sort_index().plot.bar()